<a href="https://colab.research.google.com/github/leonejesus/colab_keras_face_spoof_train/blob/main/spoof_keras_train_tl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
model_name = "siw"
database = "siw"
exp = "xp3"

in_kaggle = True
train_mode = False
test_mode = True

In [ ]:
if in_kaggle:
  from google.colab import files
  !pip install -q kaggle

  uploaded = files.upload()

  !ls
  !mkdir .kaggle
  !mv kaggle.json .kaggle
  !mv .kaggle /root/
  !kaggle datasets download -d leonejesus/$model_name-images

  !unzip $model_name-images.zip
  !rm $model_name-images.zip

In [5]:
%cd drive/My\ Drive/Colab_Notebooks/

/content/drive/My Drive/Colab_Notebooks


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers

batch_size=64

if train_mode:
  datagen = ImageDataGenerator(horizontal_flip=True, preprocessing_function=preprocess_input)
  if in_kaggle:
    disk_path="../../../"
    train_it = datagen.flow_from_directory(disk_path+model_name+'_images_train/train', class_mode='categorical', batch_size=batch_size, target_size=(224,224))
    val_it = datagen.flow_from_directory(disk_path+model_name+'_images_train/val', class_mode='categorical', target_size=(224,224))
  else:
    train_it = datagen.flow_from_directory('spoof_datasets/'+model_name+'_images/train', class_mode='categorical', batch_size=batch_size, target_size=(224,224))
    val_it = datagen.flow_from_directory('spoof_datasets/'+model_name+'_images/val', class_mode='categorical', target_size=(224,224))

if test_mode:
  datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
  if in_kaggle:
    disk_path="../../../"
    test_it = datagen.flow_from_directory(disk_path+model_name+'_images_test/test/', class_mode='categorical', target_size=(224,224), shuffle=False)
  else:
    test_it = datagen.flow_from_directory("spoof_datasets/"+database+"_images/test/", class_mode='categorical', target_size=(224,224), shuffle=False)
  test_it.reset()

Found 83200 images belonging to 2 classes.


In [7]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet',
    classes=2
    )

#for layer in conv_base.layers:
#    layer.trainable = False

x = conv_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='sigmoid', name = "extractor")(x) 
predictions = layers.Dense(2, activation='softmax')(x)
model = Model(conv_base.input, predictions)

94773248/94765736 [==============================] - 1s 0us/step


In [8]:
if train_mode:
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras.callbacks import ModelCheckpoint

  optimizer = Adam()
  model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['acc']
                )
  checkpoint_filepath = "./"+model_name+"_results/models/res50_tl.{epoch:02d}-{val_acc:.2f}_2.h5"
  model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                              save_weights_only=True,
                                              monitor='val_acc',
                                              mode='max',
                                              save_best_only=True
                                              )

In [9]:
if train_mode:
  history = model.fit(train_it,
                      steps_per_epoch=len(train_it.filepaths) // batch_size,
                      epochs=30,
                      validation_data=val_it,
                      validation_steps=len(val_it.filepaths) // batch_size,
                      callbacks=[model_checkpoint_callback]
                      )

In [10]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [11]:
if test_mode and not train_mode:

  if model_name == "rufs":
    model.load_weights(model_name+"_results/models/res50_tl.11-0.88_2.h5")
  elif model_name == "replay":
    model = load_weights(model_name+"_results/models/res50_tl.24-0.97_xp2.hdf5")
  elif model_name == "nuaa":
    model = load_weights(model_name+"_results/models/res50_tl.12-1.00_xp2.hdf5")
  elif model_name == "siw":
    model.load_weights(model_name+"_results/models/res50_tl.10-0.98_2.h5")
  elif model_name == "oulu":
    model = load_weights(model_name+"_results/models/res50_tl.09-0.98_xp2.hdf5")
  elif model_name == "rose":
    model = load_weights(model_name+"_results/models/res50_tl.21-0.97_xp2.hdf5")
  else:
    model = load_weights(model_name+"_results/models/res50_tl.14-1.00_xp2.hdf5")

In [12]:
if test_mode:
  import numpy as np

  pred = model.predict_generator(test_it, steps = len(test_it), verbose = 1)

  print(model_name+"_results/res50_tl_model_result/"+exp+"/"+database+".txt")
  file_object = open(model_name+"_results/res50_tl_model_result/"+exp+"/"+database+".txt", 'w')
  predicted_class_indices = np.argmax(pred, axis = 1)
  cont = 0
  for it in range(0, len(test_it.filepaths)):
    file_object.write("{} {} {:d} {}\n".format(predicted_class_indices[it], 
                                              pred[it][predicted_class_indices[it]],
                                              (test_it.filepaths[it].find("/spoof/") != -1),
                                              test_it.filepaths[it][test_it.filepaths[it].find("/test/")+6:]))
    if predicted_class_indices[it] == (test_it.filepaths[it].find("/spoof/") != -1):
      cont+=1
  print((cont/len(test_it.filepaths)*100))
  file_object.close()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2600/2600 [==============================] - 314s 118ms/step
siw_results/res50_tl_model_result/xp3/siw.txt
96.85697115384615
